## Oilfield Scale Detection 
  Kevin Fung

---

Following notebook is split into 3 main sections for investigating oilwell scale prediction:

### 
1. **Workflow Preparation**: Set up github communication, import modules, perform unit testing
2. **Data Processing**: Import relevant data, preprocess data, organise data for analysis
3. **Data Exploration**: Visualisation of relevant data, for gaining insight into data
4. **Data Analysis & Modelling**: Predition modelling of data, and investigations into data

In [2]:
%sh 

cwd=$(pwd)
does_my_git_repo_exist="$cwd/acse-9-independent-research-project-kkf18"

if [ -d $does_my_git_repo_exist ] 
then
  echo "Git repo exists!"
  echo "Pulling master branch..."
  cd acse-9-independent-research-project-kkf18
  git pull origin master
else
  git clone https://github.com/msc-acse/acse-9-independent-research-project-kkf18.git
  echo "Git repo does not exist!"
  echo "Cloning repo..."
fi

cd acse-9-independent-research-project-kkf18
git branch -a
git checkout Prediction_Modelling
git pull origin Prediction_Modelling

Git repo exists!
Pulling master branch...
From https://github.com/msc-acse/acse-9-independent-research-project-kkf18
 * branch master -> FETCH_HEAD
Already up-to-date.
/bin/bash: line 15: cd: acse-9-independent-research-project-kkf18: No such file or directory
* Prediction_Modelling
 master
 remotes/origin/Exploratory_Data_Analysis
 remotes/origin/HEAD -> origin/master
 remotes/origin/Prediction_Modelling
 remotes/origin/master
Already on 'Prediction_Modelling'
Your branch is up-to-date with 'origin/Prediction_Modelling'.
From https://github.com/msc-acse/acse-9-independent-research-project-kkf18
 * branch Prediction_Modelling -> FETCH_HEAD
Already up-to-date.

In [3]:
import os
import sys
import importlib.util
from pathlib import Path

# Add repo path to our sys.path for importing modules from repo.
def import_mod(module_name):
  cwd = os.getcwd()
  my_git_repo_exists = Path('{}/acse-9-independent-research-project-kkf18'.format(cwd))

  spec = importlib.util.spec_from_file_location("{}.py".format(module_name), "{}/{}.py".format(my_git_repo_exists, module_name))
  module = importlib.util.module_from_spec(spec)
  spec.loader.exec_module(module)
  
  # load module into the sys module dictionary so it can be imported in
  sys.modules[module_name] = module
  
  print("Import successful")
  
  assert module_name in sys.modules.keys()


In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import * 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

# Homemade Modules
import_mod("Data_Catalogue")
import_mod("Data_Process")
import_mod("Data_Analytics")

import Data_Catalogue as catalogue
import Data_Process as process
import Data_Analytics as model

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Import successful
Import successful
Import successful
Import successful
Import successful
Import successful

In [5]:
cwd = os.getcwd()
my_git_repo_exists = Path('{}/acse-9-independent-research-project-kkf18'.format(cwd))
sys.path.insert(0, my_git_repo_exists)

# Basic Unit Tests for Data Engineering Module
%run -i "/databricks/driver/acse-9-independent-research-project-kkf18/test_Data.py"
%run -i "/databricks/driver/acse-9-independent-research-project-kkf18/test_Data_Catalogue.py"
%run -i "/databricks/driver/acse-9-independent-research-project-kkf18/test_Data_Process.py"

Import successful
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 58108), raddr=('127.0.0.1', 39117)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 34444), raddr=('127.0.0.1', 39489)>
 self._sock = None
./databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 51040), raddr=('127.0.0.1', 44499)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 34422), raddr=('127.0.0.1', 40313)>
 self._sock = None
./databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 57254), raddr=('127.0.0.1', 40957)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 56080), raddr=('127.0.0.1', 36533)>
 self._sock = None
./databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 55074), raddr=('127.0.0.1', 46317)>
 self._sock = None

Replacing null values with zero

Replacing null values with zero
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 44506), raddr=('127.0.0.1', 46765)>
 self._sock = None
./databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 59906), raddr=('127.0.0.1', 36025)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 56122), raddr=('127.0.0.1', 35345)>
 self._sock = None
./databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 46446), raddr=('127.0.0.1', 35979)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 41082), raddr=('127.0.0.1', 42443)>
 self._sock = None
-8.074349270001139e-17 1.0
.
----------------------------------------------------------------------
Ran 6 tests in 2.868s

OK
Import successful
Import successful
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 34148), raddr=('127.0.0.1', 43443)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 49806), raddr=('127.0.0.1', 39791)>
 self._sock = None
./databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 38632), raddr=('127.0.0.1', 46865)>
 self._sock = None
/databricks/python/lib/python3.6/socket.py:657: ResourceWarning: unclosed <socket.socket fd=12, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6,

## Workbench - Data Processing

---
1. Import Production and Test Separator data into notebook
1. Construct oilwell dictionaries containing timeseries production data: `newdump_01`, `newdump_02`, `Qliq`, `interferences_filtered` for OW1, OW2, OW3
2. Construct oilwell dictionary and dataframe for the test separator data for OW1

In [7]:
# Load datas into notebook
df_01 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/newdump_01.csv')
df_02 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/newdump_02.csv')
df_OW1ql = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/Qliq.csv')
df_records = spark.read.format('csv').options(header='true', inferSchema='true', delimiter='|', encoding='iso-8859-1').load('/FileStore/tables/interences_filtered.csv')

# Dumped Data:
# Rename and cast types for each column
df_01 = df_01.select(
      df_01["Unnamed: 0"].alias("index"),
      F.to_timestamp(F.col("ts").cast("string"), "dd-MMM-yy HH:mm:ss").alias("datetime"),
      df_01["name"].alias("tag"),
      df_01["value"]
)

df_02 = df_02.select(
      df_02["Unnamed: 0"].alias("index"),
      F.to_timestamp(F.col("ts").cast("string"), "dd-MMM-yy HH:mm:ss").alias("datetime"),
      df_02["name"].alias("tag"),
      df_02["value"]
)

# Organise messy time series into sorted dictionary
organise = catalogue.DictionaryTools()
df = organise.append_data(df_01, df_02)
df = organise.removenulls("value", df)

# Tag Simplification
df_dict = organise.separate2dict(df, "tag", "datetime", "value")

r1 = re.compile('BRA-....-..-07.')
r2 = re.compile('BRA-QT  -15-0077-RAW')
OW1 = organise.separate2dict_substr(df_dict, [r1, r2])

r1 = re.compile('BRA-....-..-01.')
r2 = re.compile('BRA-QT  -15-0017-RAW')
OW3 = organise.separate2dict_substr(df_dict, [r1, r2])

r1 = re.compile('BRA-....-..-04.')
OW2 = organise.separate2dict_substr(df_dict, [r1])

# Make a tag dictionary: Decode the tags!
tag_names = {
              "BRA-PZT........" : "WHP",
              "BRA-TT..-15...." : "WHT",
              "BRA-FI........." : "GLR",
              "BRA-PT..-16...." : "GLP",
              "BRA-PT..-13...." : "DHP",
              "BRA-TT..-13...." : "DHT",
              "BRA-HV........." : "Choke",
              "BRA-QT........." : "ASD"
}

OW1 = organise.decode_keys(OW1, tag_names)
OW2 = organise.decode_keys(OW2, tag_names)
OW3 = organise.decode_keys(OW3, tag_names)


# MIKON Liquid Rate Data:
OW1["MIKON Qliq"] = df_OW1ql.select(
                                   F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy").alias("datetime"),
                                    df_OW1ql["Daily liquid rate [Sm3/d]"].alias("value")
)

OW1["MIKON Qoil"] = df_OW1ql.select(
                                   F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy").alias("datetime"),
                                    df_OW1ql["Daily oil [Sm3/d]"].alias("value")
)


# Interference Records Data:
# Translate records from norwegian to english
df_records, trans_dict = process.translate_col("Description", df_records)

# Pick out only date and description columns
OW1["Records"] = df_records.select(
                 F.to_timestamp(F.col("Date").cast("string"), "MM/dd/yyyy").alias("datetime"),
                 df_records["Description"]
)


# EXAMPLE USE CASE
# Make a dataframe with a column of matching records for timeseries data
# Recorded interferences begin 2017-01-20
OW1["WHP_2017_2019"] = OW1["WHP"].where(OW1["WHP"].datetime >= '2017-01-20')

abrv_dict = {
            "due to sand" : "Choking due to sand",
            "Valve test" : "Valve test"
}

# Match the records to the timeseries data
OW1["WHP_2017_2019"] = process.ts_overlay_records(OW1["WHP_2017_2019"], OW1["Records"], "Description", filt_dict=abrv_dict, translate_dict=trans_dict)

# Merge any two different records with matching timestamps into one record
OW1["WHP_2017_2019"] = process.merge_duplicate(OW1["WHP_2017_2019"], sqlContext)

# Add column which numerates the distinct records
OW1["WHP_2017_2019"] = process.numerate_desc(OW1["WHP_2017_2019"], "Grouped")


# Test separator data
test_separator_data = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/testseparator_1.csv')
test_OW1 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/OW1test_1.csv')

test_OW1 = test_OW1.select(F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy HH:mm").alias("datetime"), *[n for n in test_OW1.schema.names if not n == "DATE"])
test_OW1 = test_OW1.where(test_OW1.WHT != "-")
test_OW1 = test_OW1.withColumn("WHT_", test_OW1["WHT"].cast("double"))
test_OW1 = test_OW1.drop("WHT").withColumnRenamed("WHT_", "WHT")
test_OW1_dict = organise.df2dict(test_OW1)

Current samples: 865484
Appending samples: 601804
Joined samples: 1467288
/databricks/spark/python/pyspark/sql/dataframe.py:1818: UserWarning: to_replace is a dict and value is not None. value will be ignored.
 warnings.warn("to_replace is a dict and value is not None. value will be ignored.")

Collecting similar comments given the filt_dict...
/databricks/spark/python/pyspark/sql/dataframe.py:1818: UserWarning: to_replace is a dict and value is not None. value will be ignored.
 warnings.warn("to_replace is a dict and value is not None. value will be ignored.")

Overlaid records onto timeseries: You may need to remove/merge duplicates!

Duplicates found: 20669

Duplicates have been merged
/databricks/spark/python/pyspark/sql/dataframe.py:1818: UserWarning: to_replace is a dict and value is not None. value will be ignored.
 warnings.warn("to_replace is a dict and value is not None. value will be ignored.")

## Workbench - Data Exploration

---
1. Test if plot functionality works

In [9]:
# Try to find the interventions thats occurred over the years
ts_dfs = [OW1["DHP"], OW1["WHP"], OW1["MIKON Qliq"]]
ts_labels = ["DHP", "WHP", "LR"]

ge2016norm = [df.where(df.datetime >= '2016-01-11') for df in ts_dfs]

for i, df in enumerate(ge2016norm):
  mean, std = df.select(F.mean("value"), F.stddev("value")).first()
  ge2016norm[i] = df.withColumn("value_norm", (F.col("value")) / std)
  ge2016norm[i] = ge2016norm[i].select(ge2016norm[i]["datetime"], ge2016norm[i]["value_norm"].alias("value"))

fig, axs = process.plot_ts("Standardised WHP, DHP, LR over time", "datetime", "value", ge2016norm, ts_labels)
display(fig)

## Workbench - Data Analysis & Modelling

---

In [11]:
display(model.correlate(test_OW1))

In [12]:
# Lets set up a benchmark model with Linear Regression using Test Separator Data
# Note: these features are selected based on what is available both in the test separator and production data
features = ["WHP", "WHT", "Choke", "Qgl", "pbh"]

bm_dataset = model.ProcessDatasets(test_OW1)

when = lambda df : df.where((df["datetime"] >= "2018-06-01") & (df["datetime"] <= "2020-01-01"))
label_col = lambda df : df.withColumn("label", df["Q liq"])
drop_nulls = lambda df : df.select(*(["datetime"] + features + ["label"])).na.drop()

bm_dataset.apply_transformations([when, label_col, drop_nulls])
bm_X, bm_y = bm_dataset.split_df(features)
bm_y = np.squeeze(bm_y)

# Train model on the entire test separator dataset
bm_model = LinearRegression(fit_intercept=True)
bm_model.fit(bm_X, bm_y)

print("\nFitted Parameters: ")
for i, f in enumerate(features):
  print("{} : {}".format(f, bm_model.coef_[i]))

Reset dataframe to original
Dataframe has been transformed
Dataframe has been split and returned

Fitted Parameters: 
WHP : -6.071049469223777
WHT : 11.992089509625941
Choke : -5.600466922098723
Qgl : 0.0005726459415793528
pbh : -16.936937001621114

In [13]:
# Evaluate performance of Linear Regression using LOOCV
linreg = LinearRegression(fit_intercept=True)
randfor = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=1000)
print("Linear Regression")
model.kfold_scores(linreg, bm_X, np.squeeze(bm_y), k=len(bm_y))
print("\nRandom Forests")
model.kfold_scores(randfor, bm_X, np.squeeze(bm_y), k=len(bm_y))

Linear Regression

Model Evaluation
-----------------
Number of folds: 32
Averaged AIC: -6.354768504899664
Averaged RMS: 146.79284622508985

Random Forests

Model Evaluation
-----------------
Number of folds: 32
Averaged AIC: -5.713823642219003
Averaged RMS: 15.011937788046936